In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs


/Users/aman/anaconda/envs/my-rdkit-env/lib/python3.5/site-packages/rdkit/Chem/PeriodicTable.py:131: FutureWarning: split() requires a non-empty pattern match.
  splitLine = blankExpr.split(line)


In [2]:
from sklearn import linear_model

In [3]:
"""'
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
df_train.head()


,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [5]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [6]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [19]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [66]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
# smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
# df_all['smiles_len'] = pd.DataFrame(smiles_len)

# countC = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('c')))
# df_all['countC'] = pd.DataFrame(countC)

# def func():
#     for elem in df_all.smiles:
#         print elem
# #         num = elem.count('c')
# #         print num
# func()


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [8]:
def numAromatic(x):
#     moleculeString = (df_all['smiles'][1])
#     mS = moleculeString.tolist()[0]
    molecule = Chem.MolFromSmiles(str(x))
    return rdMolDescriptors.CalcNumAromaticRings(molecule)
#     print (Chem.CalcNumAromaticRings(molecule))
#     print(str(moleculeString[10:]))
#     print(type(str(moleculeString[10:])))
#     molecule = Chem.MolFromSmiles(str(moleculeString[10:]))
#     print (molecule)
#     print (Chem.lipinski.CalcNumAromaticRings(molecule))
    
#     np.vstack(df_all.smiles.astype(str).apply(lambda x: print (x)))
#     for elem in df_all.smiles.astype(str):
#         print (elem)
# #     for elem in df_all.smiles:
# #         print (elem)
# #         num = elem.count('c')
# #         print num


countAromatic = np.vstack(df_all.smiles.astype(str).apply(lambda x: numAromatic(x)))
df_all['countAromatic'] = pd.DataFrame(countAromatic)

df_all.head()


,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,countAromatic
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,6
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,3
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,5
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,4
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,7


In [ ]:

nb=64
def getBitVector(x):
    molecule = Chem.MolFromSmiles(str(x))
    feature = AllChem.GetMorganFingerprintAsBitVect(molecule, 2, nBits=nb, useFeatures=True)
    return feature

bitVector = np.vstack(df_all.smiles.astype(str).apply(lambda x:getBitVector(x)))
newFeatures=np.zeros((nb,len(bitVector)))

newFeature = []
baselineVector = bitVector[0]
for i in range(len(bitVector)):
    newFeature.append(DataStructs.DiceSimilarity(baselineVector,bitVector[1]))

df_all["similarity"] = pd.DataFrame(newFeature)


df_all.head()


In [71]:
df_all

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,C1=Cc2cnc3cc4cc(-c5scc6[nH]ccc56)c5ccccc5c4cc3...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
6,c1ncc(s1)-c1cnc2c(c1)oc1c2ccc2ccccc12,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
7,c1sc(-c2ccc3c(c2)sc2c3c3=CCC=c3c3cccnc23)c2[se...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
8,c1ccc(o1)-c1cc2cc3cc4c5c[nH]cc5ccc4cc3cc2o1,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
9,[nH]1ccc2c3c[nH]cc3c3cc(-c4cncs4)c4=CCC=c4c3c12,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [9]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print ("Train features:", X_train.shape)
print ("Train gap:", Y_train.shape)
print ("Test features:", X_test.shape)

Train features: (1000000, 257)
Train gap: (1000000,)
Test features: (824230, 257)


In [ ]:
'''LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)'''

In [13]:
def scorefunc(predictions,truth):
    sum=0
    N=len(predictions)
    for i in range(len(predictions)):
        sum+=np.square(predictions[i]-truth[i])
    ans=np.sqrt(sum/N)
    return ans

In [17]:
def runPrediction():
    
    def cross_validate(x, y, num):
        length = len(x)/num

        start = 0
        end = length
        for i in range(num):
            testData = []
            testData.append([x[start:end], y[start:end]])
            trainData = []
            trainData.append([np.concatenate([x[0:start], x[end:]]), np.concatenate([y[0:start], y[end:]])])
            
            print ("errors are " , output_errors(testData, trainData))

            start = end
            end += length
            
    def output_errors(test_array, train_array):
        result = []

        clf.fit(train_array[0][0], train_array[0][1])

        # predict data
        curPred=clf.predict(test_array[0][0])

        # add the least square error of this test data
        error = scorefunc(curPred,test_array[0][1])
        result.append(error)

        return result
    
    alphaStep = 1
    alphaParam = 0.0001
    l1_ratioParam = 0.5
    while(alphaParam < 0.1):
        clf = linear_model.Lasso(alpha=alphaParam)
#         clf2 = linear_model.LassoLars(alpha=alphaParam)
#         clf3= linear_model.Ridge(alpha=alphaParam)
#         clf4 = linear_model.ElasticNet(alpha=alphaParam, l1_ratio=l1_ratioParam)
        RF = RandomForestRegressor(n_estimators = 10)
        print (alphaParam)
        cross_validate(X_train, Y_train, 5)
        alphaParam += alphaStep 
    

runPrediction()


0.0001
errors are  [0.28438576315072323]
errors are  [0.28359367896483889]
errors are  [0.28492028821478982]
errors are  [0.28436980502478254]
errors are  [0.2851120766948535]


/Users/aman/anaconda/envs/my-rdkit-env/lib/python3.5/site-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/aman/anaconda/envs/my-rdkit-env/lib/python3.5/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [10]:
RF = RandomForestRegressor(n_estimators = 10)
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)


In [11]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [12]:
write_to_file("rkridge_latest.csv", RF_pred)
#write_to_file("sample2.csv", RF_pred)


In [ ]:

#print(clf.coef_)
#print(clf.intercept_)

In [ ]:
#write_to_file("sample3.csv",Lasso_pred)

In [ ]:
#print(Lasso_pred)

In [ ]:
#scorefunc(Lasso_pred,Y_train)

In [ ]:
#scorefunc(LR_pred,Y_train)

In [ ]:
#